使用hash和eq方法将其变成可散列对象，使用reduce实现hash，使用all和zip实现eq

In [1]:
# 使用reduce函数可以计算5!（5的阶乘）
import functools

functools.reduce(lambda a,b: a*b, range(1, 6))

120

In [2]:
# 计算整数0~5的累计异或的3种方式
n = 0

for i in range(1, 6):
    n = i
n

5

In [3]:
functools.reduce(lambda a,b: a^b, range(6))

1

In [4]:
import operator

functools.reduce(operator.xor, range(6))

1

In [ ]:
# 继续改进vector

# 通过添加len和getitem方法，来让其可切片

from array import array
import reprlib
import math
import numbers
import functools
import operator

class Vector:
    typecode = 'd'
    # _components是受保护的示例属性
    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)
    
    # 构造迭代器
    def __iter__(self):
        return iter(self._components)
    
    # 可散列
    def __hash__(self) -> int:
        # 计算各个分量的散列值
        hashes = (hash(x) for x in self._components)
        # 把hashes提供给reduce函数，使用xor函数计算聚合的散列值；第三个参数，0是初始值
        return functools.reduce(operator.xor, hashes, 0)
    
    def __eq__(self, other) -> bool:
        return len(self) == other and all(a == b for a, b in zip(self, other))
    
    # %r
    def __repr__(self) -> str:
        # 使用reprlib.repr() 函数获取self._components的有限长度表示形式（如array('d',[0.0, 1.0, 2.0, 3.0, 4.0, ...]))
        components = reprlib.repr(self._components)
        # 把字符串插入Vector的构造方法调用之前，去掉前面的array('d'和后面的)。
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    # 支持切片和索引
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        # 获取实例所属的类
        cls = type(self)
        # 如果是slice对象
        if isinstance(index, slice):
            return cls(self._components[index])
        # 如果是其他类型，就返回在这个下标的元素
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        # 获取Vector
        cls = type(self)
        #  如果属性名只有一个字母，可能是shortcut_names中的一个。
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                # 如果位置落在范围内，返回数组中对应的元素。
                return self._components[pos]
        # 如果测试都失败了，抛出AttributeError，并指明标准的消息文本。
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
    def __setattr__(self, name, value):
        cls = type(self)
        #  特别处理名称是单个字符的属性。
        if len(name) == 1:
            # 如果name是xyzt中的一个，设置特殊的错误消息。
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            # 如果name是小写字母，为所有小写字母设置一个错误消息。
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            # 否则，把错误消息设为空字符串。
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 默认情况，调用父类的setattr
        super().__setattr__(name, value)

    # 直接使用 类型代码 + self._components构建bytes对象。
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(self._components))


    # 不能使用hypot方法了，因此我们先计算各分量的平方之和，然后再使用sqrt方法开平方。
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [5]:
# 演示zip内置函数
zip(range(3), 'ABC')

In [6]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [7]:
list(zip(range(3), 'ABC', [0, 1, 2]))

[(0, 'A', 0), (1, 'B', 1), (2, 'C', 2)]

In [8]:
from itertools import zip_longest

list(zip_longest(range(3), 'ABC', [0, 1, 2, 3], fillvalue=-1))

[(0, 'A', 0), (1, 'B', 1), (2, 'C', 2), (-1, -1, 3)]